# Gather GEO Datasets
2025-01-23 ZD

Initial exploration of using Entrez NCBI E-Utilities and possibly FTP access to gather GEO dataset metadata.
Some of the INS publication gathering workflow uses E-Utilities, so there may be some overlap and copy/paste. 

This notebook and any results will not be used for production INS. Useful functions will be added as scripts to a module for use within the main workflow. 

In [1]:
import os
import sys
import json
import re

import pandas as pd
import requests
from tqdm import tqdm   # for progress bars
from Bio import Entrez  # for PubMed API


## First attempts to pull data

In [6]:
def get_full_geo_record(geo_id):
    """Get the full record for a given GEO Accession. 
    Useful for browsing the schema but not used in the production workflow.

    Args:
        geo_id (str): GEO ID from Accession (e.g. '200252411'). Note that GEO 
            Accession sometimes appears with GSE prefix which is replaced by 
            '200' in the id. 

    Returns:
        dict: Full record of data available via Entrez

    Raises:
        Exception: If an error occurs during the call. This is
            usually due to no data available
    """

    # Get user email from hidden local env file. Use default if not defined
    Entrez.email = os.environ.get('NCBI_EMAIL', 'your-email@example.com')
    Entrez.api_key = os.environ.get('NCBI_API_KEY', '')

    try:
        handle = Entrez.esummary(db="gds", id=geo_id, retmode="text")
        records = Entrez.read(handle)
        handle.close()

        # Return the full record
        return records

    except Exception as e:
        print(f"Error fetching information for GEO Accession {geo_id}: {e}")
        return None

In [3]:
full_record = get_full_geo_record('200252411')
print(json.dumps(full_record, indent=2))

[
  {
    "Item": [],
    "Id": "200252411",
    "Accession": "GSE252411",
    "GDS": "",
    "title": "Multifactoral Immune Modulation Potentiates Durable Remission in Multiple Models of Aggressive Malignancy",
    "summary": "Tumors typically lack canonical danger signals required to activate adaptive immunity and also frequently employ substantial immunomodulatory mechanisms that downregulate adaptive responses and contribute to escape from immune surveillance. Given the variety of mechanisms involved in shielding tumors from immune recognition, it is not surprising that single agent immunomodulatory approaches have been largely unsuccessful in generating durable antitumor responses. Here we report a unique combination of immunomodulatory and cytostatic agents that recondition the tumor microenvironment and eliminate complex and/or poor-prognosis tumor types including the aggressive MOC-2 model of HNSCC. scRNAseq analysis implicated up-regulation of innate immune responses and antig

In [3]:
def get_geo_ids_for_pubmed_ids(pubmed_ids: list) -> dict:
    """
    Retrieve GEO dataset IDs associated with given PubMed IDs.

    Args:
        pubmed_ids: List of PubMed IDs to query

    Returns:
        dict: Dictionary mapping PubMed IDs to their associated GEO IDs
    
    Raises:
        ValueError: If no email is configured for Entrez
        RuntimeError: If there are issues connecting to NCBI services
    """

    # Get user email from hidden local env file. Use default if not defined
    Entrez.email = os.environ.get('NCBI_EMAIL', 'your-email@example.com')
    Entrez.api_key = os.environ.get('NCBI_API_KEY', '')

    # Results dictionary to store GEO IDs for each PubMed ID
    pmid_geo_links = {}

    try:
        # Batch process PubMed IDs to reduce individual API calls
        for pmid in pubmed_ids:
            try:
                # Use ELink to find linked GEO datasets
                link_handle = Entrez.elink(
                    dbfrom="pubmed", 
                    db="gds",  # GEO database 
                    id=pmid, 
                    linkname="pubmed_gds"
                )
                
                # Read the link results
                link_record = Entrez.read(link_handle)
                link_handle.close()

                # Extract GEO IDs from the links
                geo_ids = [
                    link['Id'] 
                    for link_set in link_record 
                    for link in link_set.get('LinkSetDb', []) 
                    for link in link.get('Link', [])
                ]

                # Store results, even if empty list
                pmid_geo_links[pmid] = geo_ids

            except Exception as link_error:
                print(f"Error processing PubMed ID {pmid}: {link_error}")
                pmid_geo_links[pmid] = []

    except Exception as main_error:
        print(f"Critical error in GEO ID retrieval: {main_error}")
        raise

    return pmid_geo_links

In [5]:
pmid_list  = [
    '10637239', 
    '38738472'
]

In [6]:
pmid_geo_links = get_geo_ids_for_pubmed_ids(pmid_list)
pmid_geo_links

{'10637239': [], '38738472': ['200252411']}

In [4]:
def create_geo_dataframe(pmid_geo_links: dict) -> pd.DataFrame:
    """
    Convert PubMed to GEO ID mapping to a structured DataFrame.

    Args:
        pubmed_to_geo_mapping (dict): Mapping of PubMed IDs to GEO IDs

    Returns:
        pd.DataFrame: DataFrame with columns 'pmid' and 'geo_id'
    """
    # Prepare data for DataFrame
    df_data = []
    for pmid, geo_ids in pmid_geo_links.items():
        # If no GEO IDs, add row with NaN
        if not geo_ids:
            df_data.append({'pmid': pmid, 'geo_id': pd.NA})
        else:
            # Add a row for each GEO ID
            for geo_id in geo_ids:
                df_data.append({'pmid': pmid, 'geo_id': geo_id})
    
    # Create DataFrame
    return pd.DataFrame(df_data)

In [8]:
df = create_geo_dataframe(pmid_geo_links)
df

,pmid,geo_id
0,10637239,<NA>
1,38738472,200252411


In [9]:
df['geo_id'].nunique()

1

In [10]:
df_publications = pd.read_csv("../data/02_output/2024-09-18/gathered-2024-09-20/publication.tsv", sep="\t")
df_publications

,type,pmid,project.project_id,publication_title,authors,publication_date,cited_by,relative_citation_ratio
0,publication,10623651,P50CA062924,Immunohistochemical labeling for dpc4 mirrors ...,R E Wilentz;G H Su;J L Dai;A B Sparks;P Argani...,2000-01-01,261,4.98
1,publication,10627441,U10CA031946,Molecular analysis and clinical outcome of adu...,J L Slack;C L Willman;J W Andersen;Y P Li;D S ...,2000-01-15,49,0.88
2,publication,10627443,U24CA076518,Effect of short-term interferon therapy on the...,S Giralt;R Szydlo;J M Goldman;J Veum-Stone;J C...,2000-01-15,32,0.70
3,publication,10637239,P50CA058223,HER-2/neu amplification in benign breast disea...,A Stark;B S Hulka;S Joens;D Novotny;A D Thor;L...,2000-01-01,92,1.99
4,publication,10639144,P50CA062924,Distinct genetic profiles in colorectal tumors...,M Toyota;M Ohe-Toyota;N Ahuja;J P Issa,2000-01-18,376,6.67
...,...,...,...,...,...,...,...,...
43378,publication,39257452,U24CA258483,Non-invasive classification of IDH mutation st...,Satrajit Chakrabarty;Pamela LaMontagne;Joshua ...,2023-04-07,0,NaN
43379,publication,39260448,P50CA121974,The spike-and-slab quantile LASSO for robust v...,Yuwen Liu;Jie Ren;Shuangge Ma;Cen Wu,2024-09-11,0,NaN
43380,publication,39260448,P50CA196530,The spike-and-slab quantile LASSO for robust v...,Yuwen Liu;Jie Ren;Shuangge Ma;Cen Wu,2024-09-11,0,NaN
43381,publication,39266511,R21CA251027,Plug-and-play protein biosensors using aptamer...,Heonjoon Lee;Tian Xie;Byunghwa Kang;Xinjie Yu;...,2024-09-12,0,NaN


In [11]:
print(f"Unique Publications:        {df_publications.pmid.nunique()}")
print(f"Total Pub-Project Links:    {len(df_publications)}")

Unique Publications:        35339
Total Pub-Project Links:    43383


In [12]:
short_pmid_list = list(df_publications.pmid.unique()[0:100])

In [13]:
short_pmid_geo_links = get_geo_ids_for_pubmed_ids(short_pmid_list)

In [14]:
df_short = create_geo_dataframe(short_pmid_geo_links)
df_short

,pmid,geo_id
0,10623651,NaN
1,10627441,NaN
2,10627443,NaN
3,10637239,NaN
4,10639144,NaN
...,...,...
96,11072172,NaN
97,11075857,NaN
98,11077442,NaN
99,11085536,NaN


In [15]:
df_short[df_short['geo_id'].notna()]

,pmid,geo_id
75,10963602,200068586
76,10963602,200000061


Test of the first 100 publications only showed a single PMID (10963602) to have any GEO dataset links. (200068586 & 200000061)

## Basic functions work, but are slow. Try parallelization.

In [5]:
import time
import concurrent.futures


# Configure Entrez rate limiting
Entrez.max_tries = 3
Entrez.sleep_between_tries = 15

def get_geo_ids_for_pubmed_ids(pubmed_ids: list, api_key: bool = False) -> dict:
    """
    Retrieve GEO dataset IDs with rate limiting considerations.

    Args:
        pubmed_ids (List[str]): List of PubMed IDs to query
        api_key (bool): Whether an API key is being used

    Returns:
        Dict[str, List[str]]: Dictionary mapping PubMed IDs to their associated GEO IDs
    """
    # Configure Entrez
    Entrez.email = os.environ.get('NCBI_EMAIL', 'your-email@example.com')
    Entrez.api_key = os.environ.get('NCBI_API_KEY', '')
    
    # Configure Entrez rate limiting
    Entrez.max_tries = 3
    Entrez.sleep_between_tries = 15

    # Get counts for progress tracking
    pmid_count = len(pubmed_ids)

    # Determine appropriate number of workers based on API key
    max_workers = 10 if api_key else 3

    def fetch_geo_ids(pmid: str) -> tuple:
        """Fetch GEO IDs for a single PubMed ID with built-in rate limiting"""
        try:
            # Small delay to further control rate
            time.sleep(0.5 if not api_key else 0.1)
            
            link_handle = Entrez.elink(
                dbfrom="pubmed", 
                db="gds",  
                id=pmid, 
                linkname="pubmed_gds"
            )
            
            link_record = Entrez.read(link_handle)
            link_handle.close()

            geo_ids = [
                link['Id'] 
                for link_set in link_record 
                for link in link_set.get('LinkSetDb', []) 
                for link in link.get('Link', [])
            ]

            return (pmid, geo_ids)

        except Exception as e:
            print(f"Error processing PMID {pmid}: {e}")
            return (pmid, [])

    # Use ThreadPoolExecutor with rate-limited concurrency
    pmid_geo_links = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_geo_ids, pmid): pmid for pmid in pubmed_ids}
        
        for future in tqdm(concurrent.futures.as_completed(futures),
                            unit="PMID", total=pmid_count, ncols=80):
            pmid, geo_ids = future.result()
            pmid_geo_links[pmid] = geo_ids

    return pmid_geo_links

def create_geo_dataframe(pmid_geo_links: dict) -> pd.DataFrame:
    """
    Convert PubMed to GEO ID mapping to a structured DataFrame.

    Args:
        pmid_geo_links (Dict[str, List[str]]): Mapping of PubMed IDs to GEO IDs

    Returns:
        pd.DataFrame: DataFrame with columns 'pmid' and 'geo_id'
    """
    df_data = []
    for pmid, geo_ids in pmid_geo_links.items():
        if not geo_ids:
            df_data.append({'pmid': pmid, 'geo_id': pd.NA})
        else:
            df_data.extend([{'pmid': pmid, 'geo_id': geo_id} for geo_id in geo_ids])
    
    return pd.DataFrame(df_data)

# Combined runner function
def process_pubmed_to_geo_ids(pubmed_ids: list, max_workers: int = 10) -> pd.DataFrame:
    """
    Comprehensive function to retrieve and convert GEO IDs to DataFrame
    
    Args:
        pubmed_ids (List[str]): List of PubMed IDs to query
        max_workers (int): Maximum number of concurrent API calls

    Returns:
        pd.DataFrame: Processed DataFrame of PubMed and GEO IDs
    """
    start_time = time.time()
    
    # Retrieve GEO IDs
    geo_mapping = get_geo_ids_for_pubmed_ids(pubmed_ids, max_workers)
    
    # Convert to DataFrame
    result_df = create_geo_dataframe(geo_mapping)
    
    print(f"Total processing time: {time.time() - start_time:.2f} seconds")
    return result_df

In [17]:
df_test= process_pubmed_to_geo_ids(short_pmid_list)
df_test

100%|███████████████████████████████████████| 100/100 [00:31<00:00,  3.21PMID/s]

Total processing time: 31.20 seconds


,pmid,geo_id
0,10623651,NaN
1,10627443,NaN
2,10656455,NaN
3,10679912,NaN
4,10639144,NaN
...,...,...
96,11072172,NaN
97,11075857,NaN
98,11077442,NaN
99,11091920,NaN


In [20]:
df_test.to_csv('geo_test_first_100_pmids.csv', index=False)

In [21]:
# Run on first 5000 unique publications
pmid_0_5000 = list(df_publications.pmid.unique()[0:5000])
df_0_5000 = process_pubmed_to_geo_ids(pmid_0_5000)
df_0_5000.to_csv('geo_test_first_5000_pmids.csv', index=False)

100%|█████████████████████████████████████| 5000/5000 [33:04<00:00,  2.52PMID/s]


Total processing time: 1984.32 seconds


In [67]:
pmid_last_1000 = list(df_publications.pmid.unique()[-1000:])
df_last_1000 = process_pubmed_to_geo_ids(pmid_last_1000)
df_last_1000.to_csv('geo_test_last_1000_pmids.csv', index=False)

100%|█████████████████████████████████████| 1000/1000 [08:23<00:00,  1.99PMID/s]

Total processing time: 503.25 seconds


## We have GEO IDs from PMIDs. Now to get more GEO info

In [28]:
df_test[df_test['geo_id'].notnull()]

,pmid,geo_id
76,10963602,200068586
77,10963602,200000061


In [48]:
geo_id = 200068586
geo_id_list = [200068586,200000061]

In [62]:
get_full_geo_record(geo_id_list)

Error fetching information for GEO Accession [200068586, 200000061]: HTTP Error 500: Internal Server Error


In [ ]:
df_last_1000

In [6]:
def get_geo_raw_records(geo_id_list):
    """Get summary metadata for each GEO dataset using Esummary

    Args:
        geo_id_list: list of GEO IDs as int

    Returns:

    """

    # Build emtpy list to hold gathered records
    all_records = []

    # Iterate through list of ids to get metadata for each
    for geo_id in tqdm(geo_id_list, ncols=80):
        record = get_full_geo_record(geo_id)
        if record:
            all_records.append(record)
        else:
            print(f"Error retreiving {geo_id} metadata. Skipping.")
    
    # # Export to JSON file
    # try:
    #     with open(output_file, 'w', encoding='utf-8') as f:
    #         json.dump(all_records, f, indent=2, ensure_ascii=False)
    #     print(f"Successfully exported {len(all_records)} records to {output_file}")
    # except Exception as e:
    #     print(f"Error exporting records to JSON: {e}")

    return all_records

In [74]:
test_records = get_geo_raw_records(geo_id_list)

100%|█████████████████████████████████████████████| 2/2 [00:01<00:00,  1.91it/s]


In [76]:
print(json.dumps(test_records, indent=2))

[
  [
    {
      "Item": [],
      "Id": "200068586",
      "Accession": "GSE68586",
      "GDS": "",
      "title": "caArray_brown-00173: Molecular portraits of human breast tumours",
      "summary": "Human breast tumours are diverse in their natural history and in their responsiveness to treatments. Variation in transcriptional programs accounts for much of the biological diversity of human cells and tumours. In each cell, signal transduction and regulatory systems transduce information from the cell's identity to its environmental status, thereby controlling the level of expression of every gene in the genome. Here we have characterized variation in gene expression patterns in a set of 65 surgical specimens of human breast tumours from 42 different individuals, using complementary DNA microarrays representing 8,102 human genes. These patterns provided a distinctive molecular portrait of each tumour. Twenty of the tumours were sampled twice, before and after a 16-week course of dox

In [77]:
L1000_geo_list = df_last_1000[df_last_1000['geo_id'].notnull()]['geo_id'].unique().tolist()

In [78]:
len(L1000_geo_list)

228

In [79]:
L1000_records = get_geo_raw_records(L1000_geo_list)

100%|█████████████████████████████████████████| 228/228 [02:13<00:00,  1.71it/s]


In [80]:
with open('test_geo_records_first_1000.json', 'w', encoding='utf-8') as f:
    json.dump(L1000_records, f, indent=2, ensure_ascii=False)

## Proof of concept is there
- Use publications output to get list of pmids
- Use list of pmids to find all associated GEO ids and store as combined intermed csv
- Get raw summary metadata for each GEO id and store as intermed combined json
- Process raw metadata to build GEO df with only formatted fields of interest

### Try buildling the full GEO ID list to get expected counts

In [7]:
df_publications = pd.read_csv("../data/02_output/2024-09-18/gathered-2024-09-20/publication.tsv", sep="\t")
df_publications

,type,pmid,project.project_id,publication_title,authors,publication_date,cited_by,relative_citation_ratio
0,publication,10623651,P50CA062924,Immunohistochemical labeling for dpc4 mirrors ...,R E Wilentz;G H Su;J L Dai;A B Sparks;P Argani...,2000-01-01,261,4.98
1,publication,10627441,U10CA031946,Molecular analysis and clinical outcome of adu...,J L Slack;C L Willman;J W Andersen;Y P Li;D S ...,2000-01-15,49,0.88
2,publication,10627443,U24CA076518,Effect of short-term interferon therapy on the...,S Giralt;R Szydlo;J M Goldman;J Veum-Stone;J C...,2000-01-15,32,0.70
3,publication,10637239,P50CA058223,HER-2/neu amplification in benign breast disea...,A Stark;B S Hulka;S Joens;D Novotny;A D Thor;L...,2000-01-01,92,1.99
4,publication,10639144,P50CA062924,Distinct genetic profiles in colorectal tumors...,M Toyota;M Ohe-Toyota;N Ahuja;J P Issa,2000-01-18,376,6.67
...,...,...,...,...,...,...,...,...
43378,publication,39257452,U24CA258483,Non-invasive classification of IDH mutation st...,Satrajit Chakrabarty;Pamela LaMontagne;Joshua ...,2023-04-07,0,NaN
43379,publication,39260448,P50CA121974,The spike-and-slab quantile LASSO for robust v...,Yuwen Liu;Jie Ren;Shuangge Ma;Cen Wu,2024-09-11,0,NaN
43380,publication,39260448,P50CA196530,The spike-and-slab quantile LASSO for robust v...,Yuwen Liu;Jie Ren;Shuangge Ma;Cen Wu,2024-09-11,0,NaN
43381,publication,39266511,R21CA251027,Plug-and-play protein biosensors using aptamer...,Heonjoon Lee;Tian Xie;Byunghwa Kang;Xinjie Yu;...,2024-09-12,0,NaN


In [8]:
full_pmid_list = df_publications['pmid'].unique().tolist()
len(full_pmid_list)

35339

In [9]:
# Try the full run. Might take 2-3 hours
df_geo_pmid = process_pubmed_to_geo_ids(full_pmid_list)

  3%|█                                | 1182/35339 [09:51<24:22:15,  2.57s/PMID]

Error processing PMID 15657037: NCBI C++ Exception:
    Error: TXCLIENT(CException::eUnknown) "/pubmed_gen/rbuild/version/20240724/entrez/2.19/src/internal/txclient/TxClient.cpp", line 1045: ncbi::CTxRawClientImpl::readAll() --- Read failed: EOF (the other side has unexpectedly closed connection), peer: 130.14.22.35:8064



  8%|██▊                              | 3003/35339 [25:01<13:07:41,  1.46s/PMID]

Error processing PMID 18462436: NCBI C++ Exception:
    Error: TXCLIENT(CException::eUnknown) "/pubmed_gen/rbuild/version/20240724/entrez/2.19/src/internal/txclient/TxClient.cpp", line 1045: ncbi::CTxRawClientImpl::readAll() --- Read failed: EOF (the other side has unexpectedly closed connection), peer: 130.14.22.236:8064



100%|█████████████████████████████████| 35339/35339 [3:58:41<00:00,  2.47PMID/s]


Total processing time: 14322.07 seconds


In [10]:
df_geo_pmid.to_csv('geo_pmid_mapping.csv', index=False)

## Get additional info from FTP-accessible files

In [32]:
from ftplib import FTP
import gzip
import io
from urllib.parse import urlparse

In [29]:
full_record = get_full_geo_record(geo_id="200273401")

In [30]:
print(json.dumps(full_record, indent=2))

[
  {
    "Item": [],
    "Id": "200273401",
    "Accession": "GSE273401",
    "GDS": "",
    "title": "Valproic acid targets IDH1 mutants through alteration of lipid metabolism [2020]",
    "summary": "Histone deacetylases (HDACs) have a wide range of targets and can rewire both the chromatin and lipidome of cancer cells. In this study, we show that valproic acid (VPA), a brain penetrant anti-epilepticseizure medication and histone deacetylase inhibitor, inhibits the growth of IDH1 mutant tumors in vivo and in vitro, with at least some selectivity over IDH1 wild-type tumors. Surprisingly, genes upregulated by VPA showed no change inenhanced chromatin accessibility at the promoter, but there was a correlation between VPA-downregulated genes and diminished promoter chromatin accessibility. VPA inhibited the transcription of lipogenic genes and these lipogenic genes showed significant decreases in promoter chromatin accessibility only in the IDH1 MT glioma cell lines tested. VPA inhibite

In [31]:
ftp_link = full_record[0]['FTPLink']
print(ftp_link)

ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE273nnn/GSE273401/


In [81]:
def extract_ftp_metadata(ftp_link: str) -> dict:
    """
    Extract metadata from series matrix files given an FTP link.
    Assumes FTP link is from GEO eutilities esummary.
    
    Args:
        ftp_link (str): Full FTP link (can start with ftp:// or be a direct path)
    """
    # Clean up the FTP link
    if ftp_link.startswith('ftp://'):
        parsed = urlparse(ftp_link)
        ftp_dir = parsed.path
    else:
        ftp_dir = ftp_link
    
    # Remove any trailing slashes
    ftp_dir = ftp_dir.rstrip('/')
    
    # Ensure we're looking in the matrix subdirectory
    if not ftp_dir.endswith('/matrix'):
        ftp_dir = f"{ftp_dir}/matrix"
    
    # Connect to FTP
    ftp = FTP('ftp.ncbi.nlm.nih.gov')
    ftp.login()
    
    try:
        # Remove leading slash if present
        ftp_dir = ftp_dir.lstrip('/')
        ftp.cwd(ftp_dir)
        
        matrix_files = [f for f in ftp.nlst() if f.endswith('series_matrix.txt.gz')]
        
        if not matrix_files:
            raise FileNotFoundError(f"No series matrix files found in {ftp_dir}")
        
        metadata = {
            'contributors': [],
            'contact_name': '',
            'contact_institute': ''
        }
        
        # Process each matrix file
        for matrix_file in matrix_files:
            # Read and decompress file directly from FTP
            bio = io.BytesIO()
            ftp.retrbinary(f"RETR {matrix_file}", bio.write)
            bio.seek(0)
            
            with gzip.GzipFile(fileobj=bio, mode='rb') as gz:
                content = gz.read().decode('utf-8')
                
                # Extract fields using regex
                contributors = re.findall(r'!Series_contributor\t(.+)(?:\r\n|\r|\n)', content)
                contributors = [contributor.strip('"') for contributor in contributors]
                metadata['contributors'].extend(contributors)
                
                if not metadata['contact_name']:
                    contact_match = re.search(r'!Series_contact_name\t(.+)(?:\r\n|\r|\n)', content)
                    if contact_match:
                        metadata['contact_name'] = contact_match.group(1).strip('"')
                
                if not metadata['contact_institute']:
                    institute_match = re.search(r'!Series_contact_institute\t(.+)(?:\r\n|\r|\n)', content)
                    if institute_match:
                        metadata['contact_institute'] = institute_match.group(1).strip('"')
        
        # Remove duplicate contributors while preserving order
        metadata['contributors'] = list(dict.fromkeys(metadata['contributors']))
        return metadata
        
    finally:
        ftp.quit()

In [82]:
ftp_metadata = extract_ftp_metadata(ftp_link)

In [83]:
ftp_metadata

{'contributors': ['Lubayna,,Elahi',
  'Riki,,Kawaguchi',
  'Yue,,Qin',
  'Harley,,Kornblum'],
 'contact_name': 'Riki,,Kawaguchi',
 'contact_institute': 'University of California, Los Angeles'}

In [48]:
ftp = FTP('ftp.ncbi.nlm.nih.gov')
ftp.login()

'230 Anonymous access granted, restrictions apply'

In [49]:
ftp.cwd('geo/series/GSE273nnn/GSE273401/matrix')

'250 CWD command successful'

In [50]:
matrix_files = [f for f in ftp.nlst() if f.endswith('series_matrix.txt.gz')]
print(matrix_files)

['GSE273401_series_matrix.txt.gz']


In [51]:
for matrix_file in matrix_files:
    # Read and decompress file directly from FTP
    bio = io.BytesIO()
    ftp.retrbinary(f"RETR {matrix_file}", bio.write)
    bio.seek(0)
    
    with gzip.GzipFile(fileobj=bio, mode='rb') as gz:
        content = gz.read().decode('utf-8')

In [58]:
content

'!Series_title\t"Valproic acid targets IDH1 mutants through alteration of lipid metabolism [2020]"\n!Series_geo_accession\t"GSE273401"\n!Series_status\t"Public on Aug 21 2024"\n!Series_submission_date\t"Jul 29 2024"\n!Series_last_update_date\t"Aug 21 2024"\n!Series_pubmed_id\t"39149696"\n!Series_summary\t"Histone deacetylases (HDACs) have a wide range of targets and can rewire both the chromatin and lipidome of cancer cells. In this study, we show that valproic acid (VPA), a brain penetrant anti-epilepticseizure medication and histone deacetylase inhibitor, inhibits the growth of IDH1 mutant tumors in vivo and in vitro, with at least some selectivity over IDH1 wild-type tumors. Surprisingly, genes upregulated by VPA showed no change inenhanced chromatin accessibility at the promoter, but there was a correlation between VPA-downregulated genes and diminished promoter chromatin accessibility. VPA inhibited the transcription of lipogenic genes and these lipogenic genes showed significant 

In [68]:
print(content)

!Series_title	"Valproic acid targets IDH1 mutants through alteration of lipid metabolism [2020]"
!Series_geo_accession	"GSE273401"
!Series_status	"Public on Aug 21 2024"
!Series_submission_date	"Jul 29 2024"
!Series_last_update_date	"Aug 21 2024"
!Series_pubmed_id	"39149696"
!Series_summary	"Histone deacetylases (HDACs) have a wide range of targets and can rewire both the chromatin and lipidome of cancer cells. In this study, we show that valproic acid (VPA), a brain penetrant anti-epilepticseizure medication and histone deacetylase inhibitor, inhibits the growth of IDH1 mutant tumors in vivo and in vitro, with at least some selectivity over IDH1 wild-type tumors. Surprisingly, genes upregulated by VPA showed no change inenhanced chromatin accessibility at the promoter, but there was a correlation between VPA-downregulated genes and diminished promoter chromatin accessibility. VPA inhibited the transcription of lipogenic genes and these lipogenic genes showed significant decreases in p

In [67]:
contributors = re.findall(r'!Series_contributor\t(.+)(?:\r\n|\r|\n)', content)
contributors

['"Lubayna,,Elahi"', '"Riki,,Kawaguchi"', '"Yue,,Qin"', '"Harley,,Kornblum"']

In [115]:
test_links = {
    "200260928": "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE260nnn/GSE260928/",
    "200261852": "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE261nnn/GSE261852/",
    "200246682": "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE246nnn/GSE246682/",
    "200252002": "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE252nnn/GSE252002/",
    "200273401": "ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE273nnn/GSE273401/",
}

In [116]:
for name, link in test_links.items():
    print(name)
    print(link)

200260928
ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE260nnn/GSE260928/
200261852
ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE261nnn/GSE261852/
200246682
ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE246nnn/GSE246682/
200252002
ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE252nnn/GSE252002/
200273401
ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE273nnn/GSE273401/


In [118]:
test_dict = {}

for geo_id, ftp_link in test_links.items():
    ftp_metadata = []
    ftp_metadata = extract_ftp_metadata(ftp_link)
    test_dict[geo_id] = ftp_metadata

test_dict

{'200260928': {'contributors': ['Avishay,,Spitzer',
   'Simon,,Gritsch',
   'Masashi,,Nomura',
   'Keith,L,Ligon',
   'Daniel,P,Cahill',
   'Mario,,Suva',
   'Itay,,Tirosh'],
  'contact_name': 'Masashi,,Nomura',
  'contact_institute': 'Massachusetts General Hospital'},
 '200261852': {'contributors': ['Brandon,D,Ng',
   'Adhithi,,Rajagopalan',
   'Anastasia,I,Kousa',
   'Jacob,S,Fischman',
   'Sophia,,Chen',
   'Alyssa,,Massa',
   'Dylan,,Manuele',
   'Harold,K,Elias',
   'Michael,,Galiano',
   'Andri,L,Lemarquis',
   'Alexander,P,Boardman',
   'Susan,,DeWolf',
   'Jonah,,Pierce',
   'Bjarne,,Bogen',
   'Scott,E,James',
   'Marcel,M,van den Brink'],
  'contact_name': 'Brandon,,Ng',
  'contact_institute': 'Memorial Sloan Kettering Cancer Center'},
 '200246682': {'contributors': ['Brandon,D,Ng',
   'Adhithi,,Rajagopalan',
   'Anastasia,I,Kousa',
   'Jacob,S,Fischman',
   'Sophia,,Chen',
   'Alyssa,,Massa',
   'Dylan,,Manuele',
   'Harold,K,Elias',
   'Michael,,Galiano',
   'Andri,L,Lemarq

## Spot check some odd results

In [ ]:
# 3-digit GEO ID
record = get_full_geo_record(531)
print(json.dumps(record, indent=2))

[
  {
    "Item": [],
    "Id": "531",
    "Accession": "GDS531",
    "GDS": "531",
    "title": "Multiple myeloma and bone lesions",
    "summary": "Comparison of gene expression in bone marrow plasma cells of multiple myeloma patients with and without bone lesions. Osteolytic lesions increase in multiple myeloma patients.",
    "GPL": "8300",
    "GSE": "755",
    "taxon": "Homo sapiens",
    "entryType": "GDS",
    "gdsType": "Expression profiling by array",
    "ptechType": "",
    "valType": "count",
    "SSInfo": "disease state;disease state",
    "subsetInfo": "2 disease state",
    "PDAT": "2004/01/01",
    "suppFile": "",
    "Samples": [
      {
        "Accession": "GSM11437",
        "Title": "V5-P025"
      },
      {
        "Accession": "GSM11438",
        "Title": "V5-P029"
      },
      {
        "Accession": "GSM11439",
        "Title": "V5-P079"
      },
      {
        "Accession": "GSM11440",
        "Title": "V5-P084"
      },
      {
        "Accession": "GSM114

GEO Ids with fewer-than-usual digits still yield results in ESummary that look fine. 

In [121]:
# Many GEO IDs associated with one PMID (38326311)
many_geo_list = [
    200162068,
    200162476,
    200162477,
    200162478,
    200162479,
    200162480,
    200162481,
    200162482,
    200162483,
    200162486,
    200162488,
    200162490,
    200162575,
    200162576,
    200162609,
    200162727,
    ]

In [122]:
all_records = {}

for geo_id in many_geo_list:
    record = get_full_geo_record(geo_id)
    all_records[geo_id] = record

In [125]:
print(json.dumps(all_records, indent=2))

{
  "200162068": [
    {
      "Item": [],
      "Id": "200162068",
      "Accession": "GSE162068",
      "GDS": "",
      "title": "ASPSCR1-TFE3 reprograms transcription by organizing enhancers around hexameric VCP [HiChIP]",
      "summary": "ChIPseq experiments identify that ASPSCR1-TFE3 defines active enhancers across the genome and co-distributes with VCP/p97. RNAseq after knock-down of ASPSCR1-TFE3 or VCP, inhibition of VCP, or overexpression of each in alveolar soft part sarcoma cell lines demonstrates their co-dependence. HiChIP defines chromatin conformations that surround ASPSCR1-TFE3 targets and are lost on VCP knock-down.",
      "GPL": "24676",
      "GSE": "162068",
      "taxon": "Homo sapiens",
      "entryType": "GSE",
      "gdsType": "Other",
      "ptechType": "",
      "valType": "",
      "SSInfo": "",
      "subsetInfo": "",
      "PDAT": "2023/01/01",
      "suppFile": "BEDPE",
      "Samples": [
        {
          "Accession": "GSM4932592",
          "Title": 

In [126]:
all_records

{200162068: [{'Item': [], 'Id': '200162068', 'Accession': 'GSE162068', 'GDS': '', 'title': 'ASPSCR1-TFE3 reprograms transcription by organizing enhancers around hexameric VCP [HiChIP]', 'summary': 'ChIPseq experiments identify that ASPSCR1-TFE3 defines active enhancers across the genome and co-distributes with VCP/p97. RNAseq after knock-down of ASPSCR1-TFE3 or VCP, inhibition of VCP, or overexpression of each in alveolar soft part sarcoma cell lines demonstrates their co-dependence. HiChIP defines chromatin conformations that surround ASPSCR1-TFE3 targets and are lost on VCP knock-down.', 'GPL': '24676', 'GSE': '162068', 'taxon': 'Homo sapiens', 'entryType': 'GSE', 'gdsType': 'Other', 'ptechType': '', 'valType': '', 'SSInfo': '', 'subsetInfo': '', 'PDAT': '2023/01/01', 'suppFile': 'BEDPE', 'Samples': [{'Accession': 'GSM4932592', 'Title': 'VCP65_2 (HiChIP)'}, {'Accession': 'GSM4932593', 'Title': 'VCP67_1 (HiChIP)'}, {'Accession': 'GSM4932590', 'Title': 'FUUR1_1 (HiChIP)'}, {'Accession'

These 16 GEO IDs are all associated with a single PMID. The records are similar but not identical. Maybe different versions or updates to a long-running project? We should consider how to consolidate the display for these within INS.

## Formatting Scrapwork

In [2]:
df = pd.read_csv('geo_datasets_example.tsv', sep='\t')
df

,geo_id,pmid,coreproject,title,summary,geo_accession,n_samples,related_terms,release_date,assay_method,study_links
0,200025828,21984975,P50CA127003,Pten deficiency cooperates with KrasG12D to ac...,Almost all human pancreatic ductal adenocarcin...,GSE25828,8,Mus musculus,2011/09/01,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE25nnn...
1,4528,21984975,P50CA127003,KrasG12D pancreatic ductal epithelial cells de...,Analysis of primary pancreatic ductal epitheli...,GDS4528,8,Mus musculus,2011/09/01,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/datasets/GDS4nn...
2,200043973,22859400,P50CA058223,Age-associated gene expression in normal breas...,Background: Age is the strongest breast cancer...,GSE43973,148,Homo sapiens,2013/08/31,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE43nnn...
3,200033526,22859400,P50CA058223,Normal breast tissue of obese women is enriche...,Activation of inflammatory pathways is one pla...,GSE33526,72,Homo sapiens,2011/11/08,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE33nnn...
4,200033526,22002519,P50CA058223,Normal breast tissue of obese women is enriche...,Activation of inflammatory pathways is one pla...,GSE33526,72,Homo sapiens,2011/11/08,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE33nnn...
5,200216242,37055441,U54CA274499,Chimeric EGFR ChIP-seq of an MCF-10A clone exp...,Synthetic activation of chimeric EGFR-Erbb2 re...,GSE216242,12,Homo sapiens,2023/03/22,Genome binding/occupancy profiling by high thr...,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE216nn...
6,200214455,37055441,U54CA274499,Stochastic 10-cell profiling of an MCF-10A clo...,ErbB2 activation of MCF10A cells gives rise to...,GSE214455,72,Homo sapiens,2023/03/22,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE214nn...
7,200225767,37116489,P50CA261605,Towards a mechanistic understanding of respons...,Five year survival rates for pancreatic cancer...,GSE225767,55,Homo sapiens,2023/04/20,Expression profiling by high throughput sequen...,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE225nn...
8,200016113,22002519,P50CA058223,Activation of Host Wound Responses in Breast C...,Cancer progression is mediated by processes th...,GSE16113,121,Homo sapiens,2010/01/07,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE16nnn...


In [55]:
def transform_dataframe(df):
    """
    Transforms a dataframe by grouping rows with the same geo_id and merging their values.
    
    For 'pmid' and 'coreproject' columns, combines multiple values into semicolon-separated strings.
    For other columns, takes the first value when multiple values exist.
    
    Args:
        df: pandas DataFrame to transform
        
    Returns:
        pandas DataFrame with unique geo_ids and merged values
    """
    # Create a copy to avoid modifying the original dataframe
    result = df.copy()
    
    # Define aggregation functions for each column
    agg_funcs = {
        'pmid': lambda x: '; '.join(str(i) for i in set(x) if pd.notna(i)),
        'coreproject': lambda x: '; '.join(str(i) for i in set(x) if pd.notna(i))
    }
    
    # For all other columns, use the first value
    for col in df.columns:
        if col not in agg_funcs and col != 'geo_id':
            agg_funcs[col] = 'first'
    
    # Group by geo_id and apply the aggregation functions
    result = df.groupby('geo_id', as_index=False).agg(agg_funcs)

    # Ensure geo_id is string
    result['geo_id'] = result['geo_id'].astype(str)
    
    return result

In [56]:
df_new = transform_dataframe(df)
df_new

,geo_id,pmid,coreproject,title,summary,geo_accession,n_samples,related_terms,release_date,assay_method,study_links
0,4528,21984975,P50CA127003,KrasG12D pancreatic ductal epithelial cells de...,Analysis of primary pancreatic ductal epitheli...,GDS4528,8,Mus musculus,2011/09/01,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/datasets/GDS4nn...
1,200016113,22002519,P50CA058223,Activation of Host Wound Responses in Breast C...,Cancer progression is mediated by processes th...,GSE16113,121,Homo sapiens,2010/01/07,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE16nnn...
2,200025828,21984975,P50CA127003,Pten deficiency cooperates with KrasG12D to ac...,Almost all human pancreatic ductal adenocarcin...,GSE25828,8,Mus musculus,2011/09/01,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE25nnn...
3,200033526,22859400; 22002519,P50CA058223,Normal breast tissue of obese women is enriche...,Activation of inflammatory pathways is one pla...,GSE33526,72,Homo sapiens,2011/11/08,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE33nnn...
4,200043973,22859400,P50CA058223,Age-associated gene expression in normal breas...,Background: Age is the strongest breast cancer...,GSE43973,148,Homo sapiens,2013/08/31,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE43nnn...
5,200214455,37055441,U54CA274499,Stochastic 10-cell profiling of an MCF-10A clo...,ErbB2 activation of MCF10A cells gives rise to...,GSE214455,72,Homo sapiens,2023/03/22,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE214nn...
6,200216242,37055441,U54CA274499,Chimeric EGFR ChIP-seq of an MCF-10A clone exp...,Synthetic activation of chimeric EGFR-Erbb2 re...,GSE216242,12,Homo sapiens,2023/03/22,Genome binding/occupancy profiling by high thr...,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE216nn...
7,200225767,37116489,P50CA261605,Towards a mechanistic understanding of respons...,Five year survival rates for pancreatic cancer...,GSE225767,55,Homo sapiens,2023/04/20,Expression profiling by high throughput sequen...,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE225nn...


In [7]:
# Load FTP JSON data
raw_ftp_path = 'geo_ftp_metadata_example.json'
with open(raw_ftp_path, 'r') as f:
            ftp_metadata = json.load(f)


In [24]:
ftp_metadata.keys()

dict_keys(['200025828', '4528', '200043973', '200033526', '200216242', '200214455', '200225767', '200016113'])

In [ ]:
def select_geo_ftp_fields(data):
    """
    Extracts specific fields from GEO FTP metadata and deduplicates values
    
    Args:
        data: dict where keys are geo_ids and values are dicts of FTP data
        
    Returns:
        pandas df with columns for geo_id and extracted fields
    """
    result = {}
    
    for geo_id, record in data.items():
        result[geo_id] = {}
        
        # Fields to extract
        fields = ['Series_contact_name',
                  #'Series_contributor', 
                  #'Series_pubmed_id',
                  ]
        
        for field in fields:
            # Skip and use blank if field doesn't exist
            if field not in record:
                result[geo_id][field] = ''
                continue
                
            field_value = record[field]
            
            # Handle different data types
            if isinstance(field_value, list):
                # Remove duplicates by converting to set and back to list
                unique_values = list(set(field_value))
            elif isinstance(field_value, str):
                # Single string value
                unique_values = field_value
            else:
                # Skip fields with unexpected types
                continue
                
            result[geo_id][field] = unique_values

    # Build empty list to store data
    rows = []
    
    # Iterate through each geo_id and its data
    for geo_id, fields in result.items():
        row_data = {'geo_id': geo_id}
        
        # Process each field
        for field_name, field_value in fields.items():
            # If field value is a list, join it with semicolons
            if isinstance(field_value, list):
                row_data[field_name.lower()] = '; '.join(field_value)
            else:
                row_data[field_name.lower()] = field_value
        
        rows.append(row_data)
    
    # Create DataFrame
    df = pd.DataFrame(rows)
    
    # Ensure geo_id column exists even for empty dictionaries
    if len(df) == 0:
        df = pd.DataFrame(columns=['geo_id'])
    
    return df
    

In [ ]:
df_ftp = select_geo_ftp_fields(ftp_metadata)
df_ftp

,geo_id,series_contact_name
0,200025828,"Haoqiang,,Ying"
1,4528,
2,200043973,"Melissa,,Troester"
3,200033526,"Melissa,,Troester"
4,200216242,"Kevin,A.,Janes"
5,200214455,"Kevin,A.,Janes"
6,200225767,"Sana,D,Karam"
7,200016113,"Melissa,,Troester"


In [71]:
merged_df = pd.merge(df_new, df_ftp, how='left', on='geo_id')

In [72]:
merged_df

,geo_id,pmid,coreproject,title,summary,geo_accession,n_samples,related_terms,release_date,assay_method,study_links,series_contact_name
0,4528,21984975,P50CA127003,KrasG12D pancreatic ductal epithelial cells de...,Analysis of primary pancreatic ductal epitheli...,GDS4528,8,Mus musculus,2011/09/01,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/datasets/GDS4nn...,
1,200016113,22002519,P50CA058223,Activation of Host Wound Responses in Breast C...,Cancer progression is mediated by processes th...,GSE16113,121,Homo sapiens,2010/01/07,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE16nnn...,"Melissa,,Troester"
2,200025828,21984975,P50CA127003,Pten deficiency cooperates with KrasG12D to ac...,Almost all human pancreatic ductal adenocarcin...,GSE25828,8,Mus musculus,2011/09/01,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE25nnn...,"Haoqiang,,Ying"
3,200033526,22859400; 22002519,P50CA058223,Normal breast tissue of obese women is enriche...,Activation of inflammatory pathways is one pla...,GSE33526,72,Homo sapiens,2011/11/08,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE33nnn...,"Melissa,,Troester"
4,200043973,22859400,P50CA058223,Age-associated gene expression in normal breas...,Background: Age is the strongest breast cancer...,GSE43973,148,Homo sapiens,2013/08/31,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE43nnn...,"Melissa,,Troester"
5,200214455,37055441,U54CA274499,Stochastic 10-cell profiling of an MCF-10A clo...,ErbB2 activation of MCF10A cells gives rise to...,GSE214455,72,Homo sapiens,2023/03/22,Expression profiling by array,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE214nn...,"Kevin,A.,Janes"
6,200216242,37055441,U54CA274499,Chimeric EGFR ChIP-seq of an MCF-10A clone exp...,Synthetic activation of chimeric EGFR-Erbb2 re...,GSE216242,12,Homo sapiens,2023/03/22,Genome binding/occupancy profiling by high thr...,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE216nn...,"Kevin,A.,Janes"
7,200225767,37116489,P50CA261605,Towards a mechanistic understanding of respons...,Five year survival rates for pancreatic cancer...,GSE225767,55,Homo sapiens,2023/04/20,Expression profiling by high throughput sequen...,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE225nn...,"Sana,D,Karam"
